In [ ]:
import plotly.express as px
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 

from urllib.request import urlopen
import json

import geopandas
import ast

import sys
sys.path.append('../src/')

from utils.ckm_plotting import plot_rt, gen_dropdown

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from plotly.offline import iplot

In [ ]:
RT_DATA = '../../DATA/rt_county/rt_county.csv'

In [ ]:
rt_county_df = pd.read_csv(RT_DATA)
rt_county_df['countyFIPS'] = rt_county_df['countyFIPS'].apply(lambda x: f"{x:05d}")
#rt_county_df['date'] = pd.to_datetime(rt_county_df['date'])
rt_county_df.tail()

In [ ]:
rt_county_df['stateFIPS'] = rt_county_df['countyFIPS'].apply(lambda x: x[:2])
state_map = rt_county_df[['stateFIPS','state']].drop_duplicates().set_index('stateFIPS').to_dict()['state']

In [ ]:
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
# with open('../../DATA/geojsons/18.json') as response:
#     counties = json.load(response)
    
county_geojson_df = geopandas.read_file('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json')
county_geojson_df['STATE_NAME'] = county_geojson_df['STATE'].map(state_map)
county_geojson_df.head()

# RT Animation (by State)

In [ ]:
SAMPLE_FREQUENCY = 3# in days
DATE_SUBSET = [date for i, date in enumerate(np.sort(rt_county_df.date.unique().tolist())) if i%SAMPLE_FREQUENCY==0]

In [ ]:
def animate_state(data_df=rt_county_df,
                  geojson_df=county_geojson_df,
                  state='NY',
                  date_list=DATE_SUBSET,
                 ):
   
    rt_county_df_subset = rt_county_df[rt_county_df.date.isin(DATE_SUBSET)]
    geojson_subset = county_geojson_df[county_geojson_df.STATE_NAME==state]
    state_df = rt_county_df_subset[rt_county_df_subset['state']==state]

    fig = px.choropleth(
        data_frame=state_df, 
        #geojson=counties, 
        geojson=ast.literal_eval(geojson_subset.to_json()), 
        locations='countyFIPS', 
        color='mean',
        color_continuous_scale="Bluered",
        animation_frame='date',
        range_color=(0.2, 2),
        hover_name='region',
        labels={'mean':'R_t (mean)'},
        featureidkey="properties.id",
    )

    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

In [ ]:
fig = animate_state(state='NY')
fig.show()

# RT Dropdowns

In [ ]:
## SNAPSHOT AT A DATE
rt_county_df_subset = rt_county_df[rt_county_df.date==rt_county_df.date.max()]

STATE_LIST = np.sort(county_geojson_df.STATE_NAME.dropna().unique().tolist())
# STATE_LIST = ["NY","NJ","PA","FL", "AL"]

data = []
buttons_list = []

fig_dict = {}

for i, state in enumerate(STATE_LIST):
    
    geojson_subset = county_geojson_df[county_geojson_df.STATE_NAME==state]
    state_df = rt_county_df_subset[rt_county_df_subset['state']==state]
    
    fig = px.choropleth(
        data_frame=state_df, 
        #geojson=counties, 
        geojson=ast.literal_eval(geojson_subset.to_json()), 
        locations='countyFIPS', 
        color='mean',
        color_continuous_scale="Bluered",
        range_color=(0.2, 2),
        hover_name='region',
        labels={'mean':'R_t (mean)'},
        featureidkey="properties.id",
    )
    
    
    visible_list = [False]*len(STATE_LIST)
    visible_list[i] = True
    buttons_list.append(
        dict(label = state,
             method = 'update',
             args = [{'visible': visible_list},
                     {'title': f'{state} R(t) values'}
                    ]
            )        
    )

    fig_dict[state] = fig

    
fig_dropdown = gen_dropdown(
    figure_list=[sub_fig.to_plotly_json() for k,sub_fig in fig_dict.items()],
    button_labels=[k for k,v in fig_dict.items()],
)


data = fig_dropdown.to_plotly_json()['data']



updatemenus = list([
    dict(active=0,
         buttons=buttons_list,       
    )
])

layout = dict(
        title='Tracking R(t) values', 
        showlegend=False,
        updatemenus=updatemenus,
        geo = dict(
            fitbounds='locations',
            visible=False
              ),
        coloraxis = dict(
            cmin=0.2,
            cmax=2,
            colorscale=[[0.0, 'rgb(0,0,255)'], [1.0, 'rgb(255,0,0)']],
    )
)

fig_dropdown = dict( data=data, layout=layout )
iplot(fig_dropdown)

# RT Trends

In [ ]:
subset_df = rt_county_df[rt_county_df.state=='NY']

ncols = np.min([4, subset_df.region.nunique()])
nrows = int(np.ceil(subset_df.region.nunique() / ncols))

fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(14, nrows*3),
    sharey='row')

for ax, (county_state, result) in zip(axes.flat, subset_df.groupby('region')):
    plot_rt(county_state, result, ax)

fig.tight_layout()
fig.set_facecolor('w')